A few examples of how one can do "manual evaluation" of generative hypotheses. Not recommended.

In [ ]:
import os
import copy
import sdtools.sdexp as sdexp
import sdtools.cfg as cfg

In [ ]:
ROOT_DIR = cfg.data_dir

# Single image examples

### Steps

In [ ]:
#define specs for experiment variants
spec = {
    "exp":"A1",
    "entities":[
        {
            "finetune_path":os.path.join(ROOT_DIR,"finetune_sets/entitygirl"),
            "class_prompt":"a cell phone photo of a girl",
            "finetune_prompt":"a cell phone photo of qaeks girl",
            "n_class_img":200
        }
    ],
    "lr":2e-6,
    "n_iters":[200, 400, 800, 1200, 1600, 2400],
    "dir_model":os.path.join(ROOT_DIR,"modelexp/A1"),
    "dir_parent_classimg":os.path.join(ROOT_DIR,"class_sets"),
    "test_prompts":
        [
            "qaeks girl sits in a cornfield, smiling. Watercolor.",
            "qaeks girl licks an ice cream cone on a bench. Pencil drawing.",
            "qaeks girl hides in the corner, scared.",
            "qaeks girl lies in bed, dreaming about numbers. Digital art.",
            "qaeks girl sings into a microphone on stage, closeup."
        ]
}

In [ ]:
# run training
sdexp.run_exp(spec)

In [ ]:
for iters in spec["n_iters"]:
    sdexp.eval_model(spec, iters, n_examples=10)
    input() #pause for scoring

### Learning rate

In [ ]:
spec_base = {
    "exp":None,
    "entities":[
        {
            "finetune_path":os.path.join(ROOT_DIR,"finetune_sets/entitygirl"),
            "class_prompt":"a cell phone photo of a girl",
            "finetune_prompt":"a cell phone photo of qaeks girl",
            "n_class_img":200
        }
    ],
    "lr":None,
    "n_iters":None,
    "dir_model":None,
    "dir_parent_classimg":os.path.join(ROOT_DIR,"class_sets"),
    "test_prompts":
        [
            "qaeks girl sits in a cornfield, smiling. Watercolor.",
            "qaeks girl licks an ice cream cone on a bench. Pencil drawing.",
            "qaeks girl hides in the corner, scared.",
            "qaeks girl lies in bed, dreaming about numbers. Digital art.",
            "qaeks girl sings into a microphone on stage, closeup."
        ]
}
lst_spec = []
for exp,lr,iters in [
    ("A1A",1e-6,[1000,1500,2500]),
    ("A1B",5e-7,[1500,3000,4000]),
    ("A1C",4e-6,[200,400,800])
]:
    spec = spec_base.copy()
    spec['exp'] = exp
    spec['lr']=lr
    spec['n_iters']=iters
    spec['dir_model']=os.path.join(ROOT_DIR,f"modelexp/{exp}")
    lst_spec.append(spec)

In [ ]:
for spec in lst_spec:
    sdexp.run_exp(spec, verbose=True)

In [ ]:
for spec in lst_spec:
    for iters in spec["n_iters"]:
        sdexp.eval_model(spec, iters, n_examples=10)
        input()

# Multi-subject examples

## steps / learning rate makes a meaningful difference for multi-image training

In [ ]:
spec = {
    "exp":"A7",
    "entities":[
        {
            "finetune_path":os.path.join(ROOT_DIR,"finetune_sets/entitygirl"),
            "class_prompt":"a cell phone photo of a girl",
            "finetune_prompt":"a cell phone photo of qaeks girl",
            "n_class_img":200
        },
        {
            "finetune_path":os.path.join(ROOT_DIR,"finetune_sets/entityboy_large"),
            "class_prompt":"a cell phone photo of a boy",
            "finetune_prompt":"a cell phone photo of alskj boy",
            "n_class_img":200
        }
    ],
    "lr":1e-6,
    "n_iters":[1500, 2200, 3000, 4000],
    "dir_model":os.path.join(ROOT_DIR,"modelexp/A7"),
    "dir_parent_classimg":os.path.join(ROOT_DIR,"class_sets"),
    "test_prompts":
        [
            "qaeks girl sits in a cornfield, smiling. Watercolor.",
            "qaeks girl licks an ice cream cone on a bench. Pencil drawing.",
            "alskj boy sits in a cornfield, smiling. Watercolor.",
            "alskj boy licks an ice cream cone on a bench. Pencil drawing.",
            "qaeks girl hugs alskj boy in a desert. Oil painting.",
            "alskj boy chases qaeks girl through a field of flowers. Watercolor."
        ]
}

In [ ]:
sdexp.run_exp(spec)

In [ ]:
for iters in spec["n_iters"][1:2]:
    sdexp.eval_model(spec, iters, n_examples=10)
    input()

## Hyp: choice of subject makes a difference in the multi-image setting.

In [ ]:
spec_base = {
    "exp":None,
    "entities":[
        {
            "finetune_path":None,
            "class_prompt":None,
            "finetune_prompt":"a cell phone photo of {entity_name}",
            "n_class_img":200
        },
        {
            "finetune_path":os.path.join(ROOT_DIR,"finetune_sets/astro_square_ablationB"),
            "class_prompt":"a cell phone photo of a dog",
            "finetune_prompt":"a cell phone photo of astrmsr dog",
            "n_class_img":200
        }
    ],
    "lr":1e-6,
    "n_iters":[4500, 5000],
    "dir_model":None,
    "dir_parent_classimg":os.path.join(ROOT_DIR,"class_sets"),
    "test_prompts":
        [
            "{entity_name} hugs astrmsr dog in a desert. Oil painting.",
            "astrmsr dog chases {entity_name} through a field of flowers. Watercolor.",
            "astrmsr dog sits in {entity_name} lap on a park bench. Pencil drawing."
        ]
}

lst_spec = []
for exp, entity_name, finetune_path, class_prompt in [
    ("A9A", "entityboyc boy","finetune_sets/entityboy_ablationB","a cell phone photo of a boy"),
    ("A9B", "qaeks girl","finetune_sets/entitygirl","a cell phone photo of a girl")
]:
    spec = copy.deepcopy(spec_base)
    spec['entities'][0]['finetune_path'] = os.path.join(ROOT_DIR,finetune_path)
    spec['entities'][0]['class_prompt'] = class_prompt
    spec['entities'][0]['finetune_prompt'] = spec['entities'][0]['finetune_prompt'].format(entity_name=entity_name)
    for prompt in spec['test_prompts']:
        prompt = prompt.format(entity_name=entity_name)
    spec['exp'] = exp
    spec['dir_model'] = os.path.join(ROOT_DIR,f"modelexp/{exp}")
    spec['test_prompts'] = [prompt.format(entity_name=entity_name) for prompt in spec['test_prompts']]
    lst_spec.append(spec)

In [ ]:
for spec in lst_spec[1:]:
    # if not os.path.exists(spec['dir_model']):
    sdexp.run_exp(spec)

In [ ]:
for spec in lst_spec[1:]:
    for iters in spec["n_iters"][-1:]:
        print(iters)
        sdexp.eval_model(spec, iters, n_examples=10)
        input()